In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import beam_search

In [2]:
# juana la loca

# with open("20099-8.txt", encoding = "latin-1") as file:
#     text = file.read()
# print(len(text))
# print(text[:1000])

In [3]:
# with open("64763-0.txt", encoding = "utf-8") as file:
#     text = file.read()
# print(len(text))
# print(text[:1000])

In [4]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:100000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [5]:
# with open("seq2seq.py", encoding = "utf-8") as file:
#     text = file.read()
# print(len(text))
# print(text[:1000])

In [6]:
len(text)

556949

In [7]:
vocabulary = set(text)
len(vocabulary)

92

In [8]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [10]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']


  0%|          | 0/556929 [00:00<?, ?it/s]

torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [11]:
importlib.reload(beam_search)    
net = beam_search.Transformer(char2i, i2char, embedding_dimension = 100, encoder_layers = 2, decoder_layers = 2)
net.to(device)
# net.load_state_dict(torch.load("models/save_transformer.pt"))
# net.eval()
# pprint([net.tensor2text(t) for t in net.beam_search(source_1[:2], 
#                                                     max_predictions = 5,
#                                                     candidates = 3,
#                                                     beam_width = 4)[0][:, :, 1:]])
# net.fit(source_1[:10], target_1[:10], epochs = 50, batch_size = 150, lr = 0.0001, verbose = 3)
net.fit(source_1, target_1, epochs = 50, batch_size = 150, lr = 0.0001, verbose = 3)
# net.fit(source_2, target_2, epochs = 10, batch_size = 100, lr = 0.00001, verbose = 2)
# net.fit(source_3, target_3, epochs = 10, batch_size = 100, lr = 0.00001)

Net parameters: 1,940,887


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/3713 [00:00<?, ?it/s]

Epoch:    1, Loss: 1.5242
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' t outente', 't tutenter', ' tutenter ', 'tutenterg-', 'openterg-t']
greedy_search
[' the pawns', 't the pawn', ' the pawn ', 'the pawn t', 'ound the p']
sample
[' i groum t', 'uction bac', ' of would ', 'prove. 5. ', 'or Kt(Came']
beam_search
[['ct of the ', 't the pawn', ' the pawn ', "the King's", 'upposition'],
 [' pawn the ', 'tion the p', ' the pawns', 'the pawns ', 'upporting '],
 ['ction the ', 'tions the ', ' of the pa', 'the pawn t', 'or the paw'],
 [' the pawns', 't the King', 'ion the pa', 'the Queen ', 'ound the p'],
 [' the pawn ', 't of the p', ' of the Ki', 'the pawn a', 'ound the K']]
tensor([ -8.1500,  -8.0200,  -7.8000,  -7.8800, -10.2800], device='cuda:0')
tensor([[ -7.5500,  -8.0200,  -7.8000,  -6.8700,  -7.1400],
        [ -7.6100,  -8.2500,  -8.0600,  -7.4900,  -8.8800],
        [ -7.7700,  -8.3500,  -8.2200,  -7.8800,  -9.5000],
        [ -8.1500,  -8.400

  0%|          | 0/3713 [00:00<?, ?it/s]

...

Epoch:   48, Loss: 0.6984
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-L']
greedy_search
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t']
sample
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t']
beam_search
[['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t'],
 ['ct\nGutenbe', 't\nGutenber', '\nGutenberg', 'Gutenberg ', 'utenberg L'],
 ['ct, 1914)\n', 't, 1914)\n ', ', 1914)\n  ', 'Gutenberg\n', 'utenberg" '],
 ['ct, 1909)\n', 't, Project', "' Gutenber", 'Gutenberg"', 'utenberg\nL'],
 ['ct Guterba', 't, 1913)\n ', ', Project ', 'Gutenberg,', 'utenberg W']]
tensor([-0.2300, -0.2000, -0.1300, -0.2600, -0.1700], device='cuda:0')
tensor([[ -0.2300,  -0.2000,  -0.1300,  -0.2600,  -0.1700],
        [ -1.6000,  -1.7300,  -2.1400,  -1.6700,  -2.2800],
        [ -8.6500,  -8.6100,  -8.2900,  -4.0400,  -4.3100],
        [-10.1300, -1

  0%|          | 0/3713 [00:00<?, ?it/s]

Epoch:   49, Loss: 0.6962
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-L']
greedy_search
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t']
sample
['ct\nGutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t']
beam_search
[['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t'],
 ['ct\nGutenbe', 't\nGutenber', '\nGutenberg', 'Gutenberg ', 'utenberg L'],
 ['ct, 1914)\n', 't, 1914)\n ', ', Gutenber', 'Gutenberg\n', 'utenberg" '],
 ['cting Gute', 't, Project', ', Project ', 'Gutenberg"', 'utenberg\nL'],
 ['cting the ', 'ted Gutenb', ', 1914)\n  ', 'Gutenberg,', 'utenberg E']]
tensor([-0.2400, -0.2300, -0.2500, -0.1100, -0.0800], device='cuda:0')
tensor([[-0.2400, -0.2300, -0.2500, -0.1100, -0.0800],
        [-1.5500, -1.5800, -1.5400, -2.5000, -2.9900],
        [-8.8300, -9.4400, -8.1100, -4.3400, -4.9500],
        [-9.4700, -9.6800, -8.7000, -

  0%|          | 0/3713 [00:00<?, ?it/s]

Epoch:   50, Loss: 0.6944
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-L']
greedy_search
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t']
sample
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg-t']
beam_search
[['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg-', 'utenberg-t'],
 ['ct\nGutenbe', 't\nGutenber', '\nGutenberg', 'Gutenberg ', 'utenberg L'],
 ['ct, Projec', 't, 1914)\n ', ', 1914)\n  ', 'Gutenberg"', 'utenberg" '],
 ['ct, 1914)\n', 't, Project', ', Project ', 'Gutenberg\n', 'utenberg\nL'],
 ['ct, Gutenb', 't\nGUTENBER', "' Gutenber", 'Gutenberg,', 'utenberg\na']]
tensor([-0.1700, -0.1200, -0.1200, -0.1300, -0.1200], device='cuda:0')
tensor([[-0.1700, -0.1200, -0.1200, -0.1300, -0.1200],
        [-1.8700, -2.1600, -2.1700, -2.5500, -2.7700],
        [-8.3700, -8.5800, -8.2000, -4.1000, -4.1300],
        [-8.6400, -9.7600, -8.3200, 

In [12]:
pprint([net.tensor2text(t) for t in net.beam_search(source_1[:10], 
                                                    max_predictions = 50, 
                                                    candidates = 10,
                                                    beam_width = 10)[0][:, :, 1:]], compact = True)

RuntimeError: CUDA out of memory. Tried to allocate 30.00 MiB (GPU 0; 15.78 GiB total capacity; 5.48 GiB already allocated; 9.00 MiB free; 5.50 GiB reserved in total by PyTorch)